# 武汉房天下二手房数据爬取

利用了DrissionPage进行爬取

## 1.内容爬取

#### 1.1导入包

In [49]:
from DrissionPage import WebPage
from bs4 import BeautifulSoup
import aiohttp
import asyncio
import re
import csv
import random
import time
from typing import Generator
import time
import random
import tenacity
import pandas as pd
import os

#### 1.2全局参数配置 

In [50]:
# 配置全局参数
BASE_URL = "https://wuhan.esf.fang.com/house/i3{}/"
CSV_FILE = 'wuhan_houses.csv'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
#控制同时处理的请求数量
CONCURRENCY = 5  # 同时处理的最大并发数

#### 1.2初始化csv文件

即要保存的csv文件的内容初始化，写入表头

In [51]:
# 初始化 CSV 文件
def init_csv():
    with open(CSV_FILE, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=[
            '户型', '建筑面积', '朝向', '楼层', '装修',
            '建筑年代', '电梯', '产权性质', '住宅类别', '建筑结构',
            '建筑类别', '区域', '单价', '地铁', '地铁距离', '总价'
        ])
        writer.writeheader()

#### 1.3分页

采用yiels每次生成一个URL返回，而不是一次性的生成所有的URL，这样可以节省内存

In [52]:
# 分页生成器
def page_generator(start: int = 1, end: int = 3) -> Generator[str, None, None]:
    for page_num in range(start, end + 1):
        yield BASE_URL.format(page_num)


#### 1.4 获取HTML内容

传入URL，获取其对应的HTML内容，利用page来加载和操作网页

并利用wait.load_start()来等待网页加载完成，避免因为网络原因加载缓慢，而导致获取HTML失败

In [53]:
def get_page_html(url: str, page: WebPage) -> str:
    try:
        page.get(url)
        page.wait.load_start()  # 等待页面加载完成
        return page.html
    except Exception as e:
        print(f"页面加载失败: {str(e)}")
        return ""

#### 1.5随机延时与重试

通过随机延时，随机停止程序一段时间，模拟用户操作行为，以免被反爬虫

使用tenacity来实现重试机制，确保在网络请求失败的时候自动重试，也设置最多重试的数量，避免进入死循环

In [54]:
# 随机延时
def random_delay(min_delay=1, max_delay=5):
    time.sleep(random.uniform(min_delay, max_delay))

# 重试机制
@tenacity.retry(
    stop=tenacity.stop_after_attempt(3),  # 最多重试 3 次
    wait=tenacity.wait_fixed(2),  # 每次重试间隔 2 秒
)
async def fetch_house(session, url, semaphore):
    async with semaphore:
        try:
            async with session.get(url, timeout=30) as response:  # 超时时间设置为 30 秒
                if response.status == 200:
                    return await response.text(encoding='utf-8')  # 统一编码
                return None
        except Exception as e:
            print(f"请求失败 {url}: {str(e)}")
            raise  # 触发重试

#### 1.6动态调整解析逻辑

从HTML中提取房屋信息（标签和链接），利用BeautifulSoup，方便后续进入每个房源内检索爬取信息

In [55]:
# 动态调整解析逻辑
def extract_house_links(html: str) -> list:
    soup = BeautifulSoup(html, 'lxml')
    houses = []

    if soup.select("div.shop_list dl.clearfix"):
        for item in soup.select("div.shop_list dl.clearfix"):
            title_elem = item.select_one('span.tit_shop, a.tit_shop')
            link_elem = item.select_one('a[href]')

            if title_elem and link_elem:
                title = title_elem.get_text(strip=True)
                link = link_elem['href']

                if not link.startswith('http'):
                    link = f"https://wuhan.esf.fang.com{link}"

                houses.append({'标题': title, '链接': link})
    else:
        print("页面结构异常，跳过当前页")

    return houses

#### 1.7正则函数匹配房源信息

对单个房源信息，采用正则表达式来捕获需要的信息

In [107]:
# 正则匹配函数
def re_match(html, pattern):
    part_pattern = re.compile(pattern, re.DOTALL)
    part_match = part_pattern.search(html)
    return part_match.group(1).strip() if part_match else None

def extract_subway_info(html):
    # 正则表达式匹配地铁信息
    subway_pattern = re.compile(r'<span>\(距(\d+号线[^约]+)约(\d+)米\)</span>')
    subway_match = subway_pattern.search(html)
    if subway_match:
        # 提取地铁线路和距离
        subway_line = re.sub(r'(\d+号线).*', r'\1', subway_match.group(1).strip()) # 地铁线路（如 "1号线汉西一路站"）
        distance = subway_match.group(2).strip()     # 距离（如 "587"）
        
        # 只保留几号线（如 "1号线"）
        subway_line = re.sub(r'(\d+号线).*', r'\1', subway_line)
        
        return subway_line, f"{subway_match.group(2).strip()}米"
    else:
        # 如果没有地铁信息，返回“无”
        return "无", "无"

# 解析函数
def parse_house_detail(html, url):
    # 户型
    house = re_match(html, r'<div class="trl-item1 w146">\s*<div class="tt">\s*([^<]+?)\s*</div>\s*<div class="font14">户型</div>')
    # 建筑面积
    area = re_match(html, r'<div class="trl-item1 w182">\s*<div class="tt">\s*([^<]+?)\s*</div>\s*<div class="font14">建筑面积</div>')
    # 总价
    price = re_match(html, r'<div class="trl-item price_esf sty1">\s*<i>\s*([\d.]+)\s*</i>\s*万\s*</div>')
    # 朝向
    orientation = re_match(html,r'<div class="trl-item1 w146">\s*<div class="tt">\s*([^<]+?)\s*</div>\s*<div class="font14">朝向</div>')
    # 楼层
    # 楼层（新增精准匹配逻辑）
    floor = re_match(
        html,
        r'楼层$共\d+层$</div>\s*</div>\s*<div class="trl-item1 w182">\s*<div class="tt">\s*(?:<a[^>]*>)?([^<]+?)(?:</a>)?\s*</div>'
    ) or re_match(
        html,
        r'<div class="tt">\s*<a[^>]*class="link_rk"[^>]*>([^<]+)</a>\s*</div>\s*<div class="font14">\s*楼层'
    )
    # 装修         
    decoration = re_match(
        html,
        r'<div class="trl-item1 w132" style="border-right: 0;">\s*<div class="tt">\s*(?:<a[^>]*>)?([^<]+?)(?:</a>)?\s*</div>\s*<div class="font14">装修</div>'
    ) or re_match(
        html,
        r'<div class="tt">\s*<a[^>]*class="link_rk"[^>]*>([^<]+)</a>\s*</div>\s*<div class="font14">\s*装修'
    )
    # 建筑年代
    year = re_match(html, r'<div class="cont clearfix">.*?<span class="lab">建筑年代</span>\s*<span class="rcont">\s*([^<]+?)\s*</span>')
    # 电梯
    elevator = re_match(html,  r'<div class="cont clearfix">.*?<span class="lab">有无电梯</span>\s*<span class="rcont">\s*([^<]+?)\s*</span>')
    # 产权性质
    property_type = re_match(html, r'<span class="lab">产权性质</span>\s*<span class="rcont">\s*([^<]+?)\s*</span>')
    # 住宅类别
    housing_type = re_match(html,r'<span class="lab">住宅类别</span>\s*<span class="rcont">\s*<a[^>]*>([^<]+?)</a>')
    # 建筑结构
    residence_type = re_match(html, r'<span class="lab">建筑结构</span>\s*<span class="rcont"><a[^>]*>([^<]+)</a></span>')
    # 建筑类别
    building_category = re_match(
        html,
        r'<span class="lab">建筑结构</span>\s*<span class="rcont">\s*<a[^>]*>([^<]+?)</a>'
    ) or re_match(
        html,
        r'<span class="lab">建筑类别</span>\s*<span class="rcont">\s*<a[^>]*>([板楼|塔楼|平房]+)</a>'  # 备用匹配
    )
    # 区域
    regions_match = re.findall(
        r'<div class="lab">区&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;域</div>\s*<div class="rcont"[^>]*>(.*?)</div>',
        html,
        re.DOTALL
    )
    
    # 步骤2：清洗和标准化区域名称
    target_regions = ['硚口', '江岸', '武昌', '沌口', '东湖高新区', '洪山', '汉阳', '青山', '江汉', '蔡甸', '东西湖', '江夏', '黄陂']
    regions = []
    if regions_match:
        # 提取所有区域链接文本
        regions_raw = re.findall(r'<a[^>]*>([^<]+)</a>', regions_match[0])
        # 清洗：去除空格和冗余后缀
        regions_clean = [
            re.sub(r'\s+|(区|周边|经济开发区)$', '', r.strip())  # 删除空格及冗余后缀
            for r in regions_raw
        ]
        # 标准化：只保留目标区域名称（例如 "东湖高新区" 直接保留，"关山" 过滤掉）
        regions = [
            r for r in regions_clean
            if r in target_regions
        ]
    regions = ','.join(regions) if regions else "其他"

    # 地铁距离
    subway_line, subway_distance = extract_subway_info(html)
    # 单价
    money = re_match(html, r'<div class="trl-item1 w132" style="border-right: 0;">\s*<div class="tt">\s*([^<]+?)\s*</div>\s*<div class="font14">单价</div>')
    
    data_dict= {
        "户型": house,
        "建筑面积": area,
        "朝向": orientation,
        "楼层": floor,
        "装修": decoration,
        "建筑年代": year,
        "电梯": elevator,
        "产权性质": property_type,
        "住宅类别": housing_type,
        "建筑结构": residence_type,
        "建筑类别": building_category,
        "区域": regions,
        "单价": money,
        "地铁": subway_line,       
        "地铁距离": subway_distance,
        "总价": price
    }
    return data_dict


#### 1.8 异步请求

使用信号量控制并发请求数量，发送异步HTTP的请求并控制时间，然后获取HTML的内容，调用异步解析函数提取房屋详情数据

In [57]:
# 异步请求函数
async def fetch_house(session: aiohttp.ClientSession, url: str, semaphore: asyncio.Semaphore):
    async with semaphore:
        try:
            async with session.get(url, timeout=10) as response:
                if response.status == 200:
                    html = await response.text()
                    return parse_house_detail(html, url)
                return None
        except Exception as e:
            print(f"请求失败 {url}: {str(e)}")
            return None

####  1.9 爬虫入口主函数

包括了以上的各个模块工作的调用

In [108]:
async def main():
    # 初始化统计变量
    total_records = 0          # 总记录数
    total_valid_fields = 0     # 有效字段总数
    total_fields_per_record = 16  # 每条记录有 16 个字段
    init_csv()
    page = WebPage()
    page.set.headers(HEADERS)  # 正确设置请求头
    semaphore = asyncio.Semaphore(CONCURRENCY)

    async with aiohttp.ClientSession(headers=HEADERS) as session:
        ext_data=pd.read_csv("wuhan_houses.csv", encoding='utf-8-sig')
        
        # 分页处理
        for page_url in page_generator(44, 44):
            print(f"正在处理分页: {page_url}")
            html = get_page_html(page_url, page)

            if not html:
                continue

            house_links = extract_house_links(html)
            print(f"本页找到 {len(house_links)} 条房源")

            # 批量处理当前页房源
            tasks = [fetch_house(session, house['链接'], semaphore) for house in house_links]

            
            
            # 分批处理避免内存过大
            batch_size = 10
            for i in range(0, len(tasks), batch_size):
                batch = tasks[i:i + batch_size]
                results = await asyncio.gather(*batch)

                # 写入CSV
                with open(CSV_FILE, 'a', newline='', encoding='utf-8-sig') as f:
                    writer = csv.DictWriter(f, fieldnames=[
                        '户型', '建筑面积', '朝向', '楼层', '装修',
                        '建筑年代', '电梯', '产权性质', '住宅类别', '建筑结构',
                        '建筑类别', '区域', '单价', '地铁', '地铁距离', '总价'
                    ])
                    for result in filter(None, results):
                        # 统计有效字段数
                        valid_fields = sum(1 for value in result.values() if value not in [None, "无"])
                        total_valid_fields += valid_fields
                        total_records += 1

                        writer.writerow(result)

                print(f"已保存 {len(results)} 条数据")

            # 分页间延时
            time.sleep(random.uniform(5, 10))
        # 计算全局结构化存储率
    if total_records > 0:
        global_storage_rate = total_valid_fields / (total_records * total_fields_per_record)
        print(f"全局结构化存储率: {global_storage_rate:.2%}")

In [ ]:
# Jupyter专用运行方式
if __name__ == "__main__":
    # 在Jupyter中运行需要这行
    # import nest_asyncio; nest_asyncio.apply()
    await main()

## 2检查爬取内容

检查爬取内容是否有错误，以及爬取数据的形状

In [110]:
data=pd.read_csv("wuhan_houses.csv", encoding='utf-8-sig')
data.head()

,户型,建筑面积,朝向,楼层,装修,建筑年代,电梯,产权性质,住宅类别,建筑结构,建筑类别,区域,单价,地铁,地铁距离,总价
0,2室1厅1卫,64.48平米,东南,低层,精装修,2006年,有,商品房,普通住宅,平层,板楼,硚口,13958元/平米,1号线,587米,90.0
1,2室2厅1卫,64.1平米,西南,中层,精装修,NaN,有,商品房,普通住宅,平层,板楼,江岸,16069元/平米,8号线,946米,103.0
2,4室2厅2卫,147平米,南北,高层,精装修,2017年,有,个人产权,个人产权,平层,板楼,硚口,12245元/平米,1号线,532米,180.0
3,1室0厅1卫,27.56平米,南,中层,精装修,NaN,无,个人产权,个人产权,平层,NaN,江岸,30842元/平米,1号线,375米,85.0
4,3室2厅1卫,92.13平米,东南,顶层,精装修,NaN,有,个人产权,个人产权,平层,板楼,硚口,12482元/平米,1号线,468米,115.0


In [61]:
data["产权性质"].unique()

array(['商品房', '个人产权', nan, '经济适用房'], dtype=object)

In [62]:
data["住宅类别"].unique()

array(['普通住宅', '个人产权', '独栋', '平层', '公寓', '经济适用房', nan, '双拼', '联排', '叠加'],
      dtype=object)

In [63]:
data["区域"].unique()

array(['硚口', '江岸', '武昌', '沌口', '东湖高新区', '洪山', '汉阳', '青山', '江汉', '蔡甸',
       '武汉周边', '东西湖', '江夏', '黄陂', '经济开发区', '汉南', nan], dtype=object)

In [64]:
data["装修"].unique()

array(['精装修', '简装修', '毛坯', '中装修', '豪华装修', nan], dtype=object)

In [65]:
data["楼层"].unique()

array(['低层', '中层', '高层', '顶层', '底层', nan], dtype=object)

In [66]:
data["建筑类别"].unique()

array(['板楼', nan], dtype=object)

In [67]:
data["地铁"].unique()

array(['1号线', '8号线', '无', '5号线', '7号线', '19号线', '11号线', '2号线', '6号线',
       '4号线', '3号线', '21号线', '16号线'], dtype=object)

In [115]:
data.shape

(11067, 16)